In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
import time
from pyspark.sql import SparkSession

/tmp/ipykernel_58440/3760276594.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
spark = SparkSession.builder \
       .master("local[*]") \
       .appName("test")\
       .config("spark.driver.extraClassPath", "/dados10t/datalake/raw/omop/docker_imagens/postgresql-42.6.0.jar")\
       .config("spark.driver.memory", "80g")\
       .config("spark.executor.memory", "50g")\
       .config("spark.dirver.maxResultSize", "50g")\
       .config('spark.local.dir', '/dados10t/datalake/raw/checkpoint_dir')\
       .getOrCreate()
sc = spark.sparkContext
sc.setCheckpointDir('/dados10t/datalake/raw/checkpoint_dir')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/20 23:34:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/20 23:34:10 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).
23/10/20 23:34:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/10/20 23:34:11 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/10/20 23:34:11 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


#### reading table

In [3]:
df = spark.read.parquet('/dados10t/datalake/standard/omop/tabelas_omop_cmd_5.4_v1/tb_condition_occurence_data.parquet/')

In [4]:
df.printSchema()

root
 |-- visit_occurrence_id: integer (nullable = true)
 |-- condition_start_date: date (nullable = true)
 |-- condition_concept_id: string (nullable = true)
 |-- person_id: integer (nullable = true)
 |-- condition_type_concept_id: integer (nullable = true)
 |-- condition_occurrence_id: integer (nullable = true)



In [9]:
df.show()

+-------------------+--------------------+--------------------+---------+-------------------------+-----------------------+
|visit_occurrence_id|condition_start_date|condition_concept_id|person_id|condition_type_concept_id|condition_occurrence_id|
+-------------------+--------------------+--------------------+---------+-------------------------+-----------------------+
|          478617188|          2021-10-03|              321588| 37733095|                    32817|                      1|
|          478618006|          2021-12-19|              254761| 37734049|                    32817|                      2|
|          478618374|          2021-11-15|              254761| 37734455|                    32817|                      3|
|          478618374|          2021-11-15|             4186898| 37734455|                    32817|                      4|
|          478618374|          2021-11-15|              312437| 37734455|                    32817|                      5|
|       

In [5]:
df = df.withColumn('condition_concept_id', F.col('condition_concept_id').cast(IntegerType()))

In [4]:
options = {
    "url": "jdbc:postgresql://157.86.211.201:5432/postgres",
    "dbtable": "omop54_v2.condition_occurrence",
    "user": "postgres",
    "password": "omop2023",
    "postgreSQLConf": "constraint.mode=noconstraints"
}

In [9]:
df = df.filter(F.col('condition_start_date').isNotNull())

In [5]:
df.printSchema()

root
 |-- visit_occurrence_id: integer (nullable = true)
 |-- condition_start_date: date (nullable = true)
 |-- condition_concept_id: integer (nullable = true)
 |-- person_id: integer (nullable = true)
 |-- condition_type_concept_id: integer (nullable = true)
 |-- condition_occurrence_id: integer (nullable = true)



In [6]:
df.write \
    .format("jdbc") \
    .options(**options) \
    .mode("append") \
    .save()

In [7]:
print('terminou')

terminou
